In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_384299/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 16:37:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/14 16:37:44 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
import pandas as pd

# Sample data for UserActivity table
data = {
    'username': ['Alice', 'Alice', 'Alice', 'Bob'],
    'activity': ['Travel', 'Dancing', 'Travel', 'Travel'],
    'startDate': ['2020-02-12', '2020-02-21', '2020-02-24', '2020-02-11'],
    'endDate': ['2020-02-20', '2020-02-23', '2020-02-28', '2020-02-18']
}

# Convert date strings to datetime objects
for col in ['startDate', 'endDate']:
    data[col] = pd.to_datetime(data[col])

# Create DataFrame
df = pd.DataFrame(data)


df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("UserActivity")



In [5]:
df

,username,activity,startDate,endDate
0,Alice,Travel,2020-02-12,2020-02-20
1,Alice,Dancing,2020-02-21,2020-02-23
2,Alice,Travel,2020-02-24,2020-02-28
3,Bob,Travel,2020-02-11,2020-02-18


In [18]:
query = """
    select
        username,
        activity,
        startDate,
        endDate
    from (
        select
            *,
            case
                when (rk = 2) or (rk = 1 and lead(rk) over(partition by username order by rk asc) is null ) then 1
                else 0
            end as take
        from (
            select
                *,
                dense_rank() over (partition by username order by startDate desc) as rk
                
            from UserActivity u
        )
    )
        where take = 1
"""

In [19]:
spark.sql(query).show()

+--------+--------+-------------------+-------------------+
|username|activity|          startDate|            endDate|
+--------+--------+-------------------+-------------------+
|   Alice| Dancing|2020-02-21 00:00:00|2020-02-23 00:00:00|
|     Bob|  Travel|2020-02-11 00:00:00|2020-02-18 00:00:00|
+--------+--------+-------------------+-------------------+

